# Convert Excel Spreadsheet To JSONSchema Forms

In [5]:
import json
import pandas
import os
import re

SHEET_PATH = '/Users/dedan/tmp/OU.OrangutanName.S.1.2017.xls'
FORMS_PATH = '/Users/dedan/projects/monkey-db/test/test-forms/'
DATA_PATH = '/Users/dedan/projects/monkey-db/test/test-folder/'
PER_PAGE_VARIABLES = ['source', 'pg#', 'entry#', 'dataENTRYdate', 'dataENTRYperson']
all_sheets = pandas.read_excel(SHEET_PATH, sheetname=None)

## Index to Definitions

Read the index spreadsheet and clean it up.

In [6]:
index = all_sheets['INDEX']
cols_to_use = [
    'Data variable as written on entry form',
    'Excel column header',
    'Data values as written on entry form',
    'Data values to enter into excel',
    'Which forms?',
]
cols_to_join = ['Data variable as written on entry form', 'Unnamed: 1', 'Unnamed: 2', 'Unnamed: 3']
def joiner(row):
    bla = [s for s in row if isinstance(s, str) and s]
    return ' - '.join(bla) if bla else row.iloc[0]
index['Data variable as written on entry form'] = index[cols_to_join].apply(joiner, axis=1)

index = index[cols_to_use].reset_index()
index.columns = ['group', 'form_variable', 'variable', 'form_value', 'value', 'used_on']
index.fillna(method='pad', inplace=True)
index[:10]

,group,form_variable,variable,form_value,value,used_on
0,ID,Orangutan Name,ouname,name,[name],N-AR
1,ID,Jenis kelamin / Sex,sex,♀,F,N-AR
2,ID,Jenis kelamin / Sex,sex,♂,M,N-AR
3,ID,Jenis kelamin / Sex,sex,[blank],Ø,N-AR
4,General,Tanggal / Date,date,date,dd-MMM-yyyy,"F-AA, F-AN, F-BM, F-KF, F-GE, F-HTT, F-KU, F-T..."
5,General,Tanggal / Date,date,[blank],Ø,"F-AA, F-AN, F-BM, F-KF, F-GE, F-HTT, F-KU, F-T..."
6,General,Examiner,examiner,[as written],[as written],"F-AA, F-KF, F-GE, F-HTT,"
7,General,Examiner,examiner,[blank],Ø,"F-AA, F-KF, F-GE, F-HTT,"
8,General,Location,loc,Camp Mentawa,KL.CME,"F-AA, F-GE, F-HTT"
9,General,Location,loc,Camp Rendell,KL.CRE,"F-AA, F-GE, F-HTT"


Compute JSONSchema definitions from the index.

In [7]:
def grouper(g):
    return {
        'title': str(g.form_variable.iloc[0]),
        'enum': list(g.value),
        'enumNames': list(g.form_value),
        'type': 'string',
    }
definitions = index.groupby(['form_variable', 'variable']).apply(grouper)
definitions = definitions.reset_index(name='type_def')
# TODO: Why do we have duplicates?
definitions = definitions.drop_duplicates(subset=['variable'])

definitions_schema = {
    row.variable: row.type_def
    for (index, row) in definitions.iterrows()
}

## Excel Sheet to JSONSchema Form

In [8]:
all_form_names = list(all_sheets.keys() - ['INDEX'])

def get_form_for_sheet(form_name, form_sheet, definitions_schema):
    form_variables = form_sheet.columns[len(PER_PAGE_VARIABLES):]

    form_definitions = {
        var:definition 
        for var, definition in definitions_schema.items() 
        if var in form_variables
    }
    # TODO: Why do we have variables without definitions?
    form_variables = [var for var in form_variables if var in form_definitions]
    return get_schema(form_name, form_definitions, form_variables)

def read_form_sheet(spreadsheet_path, all_sheets, sheet_name):
    df = all_sheets[sheet_name]
    mask = df['Information about data Entry'] == 'source'
    header_index = df[mask].index[0] + 1
    return pandas.read_excel(spreadsheet_path, sheetname=sheet_name, header=header_index)
    
def get_schema(form_name, form_definitions, form_variables):
    return {
        'type': 'object',
        'title': form_name,
        'definitions': form_definitions,
        'properties': {
            var: {
                '$ref': '#/definitions/' + var
            } for var in form_variables
        }
    }

for form_name in all_form_names:
    form_sheet = read_form_sheet(SHEET_PATH, all_sheets, form_name)
    form = get_form_for_sheet(form_name, form_sheet, definitions_schema)
    with open(os.path.join(FORMS_PATH, form_name + '.json'), 'w') as f:
        json.dump(form, f)

## Export Entries to JSON

In [9]:
page_pattern = re.compile('(\d+)(\w?)')
def parse_page_string(page_string):
    match = page_pattern.match(str(page_string))
    page_number = int(match.group(1))
    page_char = match.group(2)
    return 'p{0:03d}{1:}'.format(page_number, page_char)

total = 0
for form_name in all_form_names:
    
    form_sheet = read_form_sheet(SHEET_PATH, all_sheets, form_name)
    entry_columns = form_sheet.columns[3:]
    for index, row in form_sheet.iterrows():
        
        try:
            # TODO: Normalize book to upercase orangname and generally validate against regex.
            book = row.source.split('_')[0]
            page = parse_page_string(row['pg#'])
            entry = str(row['entry#'])
            entryFileName = '{}_{}.json'.format(entry, form_name)            
            entry_path = os.path.join(DATA_PATH, book, page, entryFileName)
            os.makedirs(os.path.dirname(entry_path), exist_ok=True)
            # TODO: Validate with shema before writing.
            with open(entry_path, 'w') as f:
                total = total + 1
                f.write(row[entry_columns].to_json())
        except Exception as e: 
            print(form_name)
            if 'pgQ' in row:
                print('Wrong page column name')
            else:
                print('Invalid Entry (book: {}, page: {}, entry: {})'.format(row['source'], row['pg#'], row['entry#']))
                
print(total)        

N-BSC
Invalid Entry (book: OU.Ruth.F.1.2007, page: nan, entry: nan)
F-PM
Wrong page column name
F-PM
Wrong page column name
N-MD
Invalid Entry (book: nan, page: nan, entry: nan)
N-MD
Invalid Entry (book: nan, page: nan, entry: nan)
N-MD
Invalid Entry (book: nan, page: nan, entry: nan)
F-GE
Invalid Entry (book: nan, page: nan, entry: nan)
F-GE
Invalid Entry (book: nan, page: nan, entry: nan)
F-GE
Invalid Entry (book: nan, page: nan, entry: nan)
F-GE
Invalid Entry (book: nan, page: nan, entry: nan)
815
